In [3]:
import h5py
import nibabel as nib
import numpy as np

def fibers_to_nifti(fibers, voxel_size=1.0, output_path='DRTT.nii.gz'):
    x_coords, y_coords, z_coords = fibers[0], fibers[1], fibers[2]
    x_min, x_max = np.floor(x_coords.min()), np.ceil(x_coords.max())
    y_min, y_max = np.floor(y_coords.min()), np.ceil(y_coords.max())
    z_min, z_max = np.floor(z_coords.min()), np.ceil(z_coords.max())
    grid_shape = (
        int((x_max - x_min) / voxel_size) + 1,
        int((y_max - y_min) / voxel_size) + 1,
        int((z_max - z_min) / voxel_size) + 1,
    )
    voxel_grid = np.zeros(grid_shape, dtype=np.uint8)
    voxel_indices = np.vstack([
        ((x_coords - x_min) / voxel_size).astype(int),
        ((y_coords - y_min) / voxel_size).astype(int),
        ((z_coords - z_min) / voxel_size).astype(int),
    ]).T
    for idx in voxel_indices:
        voxel_grid[tuple(idx)] = 1
    affine = np.eye(4)
    affine[:3, :3] *= voxel_size
    affine[:3, 3] = [x_min, y_min, z_min]
    nifti_img = nib.Nifti1Image(voxel_grid, affine)
    nib.save(nifti_img, output_path)
    return nifti_img

In [ ]:
# Load the fibers data
with h5py.File('/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/MATLAB/leaddbs/templates/space/MNI_ICBM_2009b_NLIN_ASYM/atlases/Brainstem Connectome (Meola 2016)/rh/DRTT.mat', 'r') as mat_file:
    fibers_data = mat_file['fibers'][...]

# Convert fibers to NIfTI
fibers_to_nifti(fibers_data, voxel_size=1.0, output_path='/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/tracts/dbs/rh_DRTT.nii.gz')

If You Made Individual LH and RH files, combine them

In [5]:
import nibabel as nib
import numpy as np
import os

# Paths to the left and right DRTT NIfTI files
lh_path = "/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/tracts/dbs/lh_DRTT.nii.gz"
rh_path = "/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/tracts/dbs/rh_DRTT.nii.gz"

# Output path for the combined file
output_path = "/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/atlases/tracts/dbs/combined_DRTT.nii.gz"

# Load the left and right NIfTI images
lh_img = nib.load(lh_path)
rh_img = nib.load(rh_path)

# Extract the data arrays
lh_data = lh_img.get_fdata()
rh_data = rh_img.get_fdata()

# Check that the dimensions match
if lh_data.shape != rh_data.shape:
    print(lh_data.shape, rh_data.shape)
    raise ValueError("Left and right DRTT tracts have different dimensions and cannot be added directly.")

# Combine the two tracts by adding their data
combined_data = lh_data + rh_data

# Use the affine from one of the inputs (assuming they are in the same space)
combined_affine = lh_img.affine

# Create a new NIfTI image
combined_img = nib.Nifti1Image(combined_data, combined_affine)

# Save the combined image
nib.save(combined_img, output_path)

print(f"Combined DRTT tract saved to: {output_path}")


(41, 65, 44) (42, 68, 49)


ValueError: Left and right DRTT tracts have different dimensions and cannot be added directly.